# Day 1 - Examples: Classification

In this notebook we offer an example of classification via prompting. That is, given some text, we ask the model to classify the text into things like topics or sentiment. 

References/Further Reading:

* This tutorial is partially based on https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_classification.ipynb

In [2]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv("../../.env")

True

# News Classification

## Setup template

We setup a template here telling the model to classify news headlines into one of 5 categories. Notice that we have provided the model with a number of examples of what the output should look like. This is called *few-shot prompting*! This helps the model to "understand" the kind of answer that we want from it.

Feel free to modify this to your own needs!

In [3]:
selected_article = "<article>"
template_prompt = f"""
Classify the given news headlines into one of the following categories: [business, entertainment, health, sports, technology]

Text: Pixel 7 Pro Expert Hands On Review. 
The answer is: technology 

Text: Quit smoking? 
The answer is: health 

Text: Birdies or bogeys? Top 5 tips to hit under par 
The answer is: sports 

Text: Relief from local minimum-wage hike looking more remote 
The answer is: business 

Text: {selected_article} 
The answer is: 
"""

print(template_prompt)


Classify the given news headlines into one of the following categories: [business, entertainment, health, sports, technology]

Text: Pixel 7 Pro Expert Hands On Review. 
The answer is: technology 

Text: Quit smoking? 
The answer is: health 

Text: Birdies or bogeys? Top 5 tips to hit under par 
The answer is: sports 

Text: Relief from local minimum-wage hike looking more remote 
The answer is: business 

Text: <article> 
The answer is: 



#### User Input

Here we add in the document we want the model to read. Feel free to modify this to a document of your choice!

In [4]:
selected_article = "Introducing Apple Vision Pro: Apple’s first spatial computer"

## OpenAI

As you might expect by now, it's just a matter of sending your prompt to the model!

In [94]:
import openai

prompt = template_prompt.replace('<article>', selected_article)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

chatbot_response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  temperature=1,
  max_tokens=1500,
)

print(chatbot_response.choices[0].message["content"])

technology


## Google

You know the drill now!

In [95]:
from vertexai.preview.language_models import TextGenerationModel

prompt = template_prompt.replace('<article>', selected_article)

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(prompt, max_output_tokens=1024)
print(response.text)

technology


## Open Source LLM: Falcon-7B

In [9]:
import requests
import os

prompt = template_prompt.replace('<article>', selected_article)

headers = {"Authorization": f"Bearer {os.environ.get('HUGGINGFACEHUB_API_TOKEN')}"}

def query(payload):
    response = requests.request("POST", os.environ.get("HUGGINGFACEHUB_ENDPOINT"), headers=headers, json=payload)
    return response.json()

data = query({"inputs": prompt, "parameters": {"max_new_tokens": 10, "return_full_text": False, "top_k": 10, "temperature": 1}})

print(data[0]['generated_text'])


Technology


Similar to the previous examples, we had to fiddle with the parameters to get Falcon to work well.

# Sentiment Classification

Let's consider sentiment classification as another example. Given some text, we need to determine if the text is positive, negative or neutral.

## Setup template

This time we setup a zero-shot prompting example to classify review sentiment.

Feel free to modify this to your own needs!

In [10]:
selected_article = "<article>"
template_prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative".

Text: {selected_article} 
The answer is: 
"""

print(template_prompt)

Classify the sentiment of the following review as "positive", "neutral" or "negative".

Text: <article> 
The answer is: 



#### User Input

Here we add in the document we want the model to read. Feel free to modify this to a document of your choice!

In [11]:
selected_article = "I loved the new Spider-Man movie!! The animation was really fluid"

## OpenAI

In [112]:
import openai

prompt = template_prompt.replace('<article>', selected_article)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

chatbot_response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  temperature=1,
  max_tokens=1500,
)

print(chatbot_response.choices[0].message["content"])

positive


## Google

In [113]:
from vertexai.preview.language_models import TextGenerationModel

prompt = template_prompt.replace('<article>', selected_article)

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(prompt, max_output_tokens=1024)
print(response.text)

positive


## Open Source LLM: Falcon-7B

In [14]:
import requests
import os

prompt = template_prompt.replace('<article>', selected_article)

headers = {"Authorization": f"Bearer {os.environ.get('HUGGINGFACEHUB_API_TOKEN')}"}

def query(payload):
    response = requests.request("POST", os.environ.get("HUGGINGFACEHUB_ENDPOINT"), headers=headers, json=payload)
    return response.json()

data = query({"inputs": prompt, "parameters": {"max_new_tokens": 250, "return_full_text": False}})

print(data[0]['generated_text'])


Positive
